In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = os.getcwd() + "/image_data"
path

In [ ]:
TRAINING_DIR = path+"/train"
VALIDATION_DIR = path + "/val"
TEST_DIR = path + "/test"

print(TRAINING_DIR)
print(VALIDATION_DIR)
print(TEST_DIR)

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR, TEST_DIR):
  train_datagen = ImageDataGenerator(rescale=1.0/255)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=128,
                                                      class_mode="binary",
                                                      target_size=(224, 224))

  validation_datagen = ImageDataGenerator(rescale=1.0/255)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=128,
                                                                class_mode="binary",
                                                                target_size=(224, 224))
  test_datagen = ImageDataGenerator(rescale=1.0/255)
  
  test_generator = test_datagen.flow_from_directory(directory=TEST_DIR,
                                                    batch_size=128,
                                                    class_mode="binary",
                                                    target_size=(224, 224))
  return train_generator, validation_generator, test_generator

In [ ]:
train_generator, validation_generator, test_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR, TEST_DIR)

class_names = train_generator.class_indices
print(class_names)

In [ ]:
img1, img2 = train_generator[0][:2]

In [ ]:
# fig, ax = plt.subplots(2,3, figsize=(10,8))

# ax[0].plot(img1, x_axis='time')

In [ ]:
model = keras.Sequential([
    layers.Conv2D(16, (3,3), activation = "relu", input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation = "relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation = "relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation = "relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(256, (3,3), activation = "relu"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(512, (3,3), activation = "relu"),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(512, activation = "relu"),
    layers.Dropout(0.2),
    layers.Dense(512, activation = "relu"),    
    layers.Dense(1, activation = "sigmoid")               
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# tf.keras.losses.BinaryCrossentropy()
model.summary()

In [ ]:
earlystopper = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

In [ ]:
epochs = 23
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks = [earlystopper]
)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

In [ ]:
test_predict = model.predict(test_generator)

In [ ]:
show_test_predict = np.round(test_predict, 0)
show_test_predict = pd.DataFrame(show_test_predict)
print(show_test_predict)
print(len(show_test_predict))

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(test_generator.classes, test_predict)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)'% roc_auc)
plt.plot([0,1],[0,1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import f1_score
f1_score(test_generator.classes, show_test_predict, average='macro')

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_generator.classes, show_test_predict)

In [ ]:
model.evaluate(test_generator)

In [ ]:
# os.chdir("D:\\Study\\Code\\Python\\2023Sound_Classification_Bee_Qeen_Queenless\\Models")
model.save("CNN.h5")

In [ ]:
# STFT, Chroma
# C1:using wavelet --> Feature selection
# C2: MFCC --> spectral --> vector(using average) --> feature selection --> ML(KNN, SVM, Logistic)
# C3: HHT --> spectral --> average --> HHT 
# ==> Apply DL, ML
## => Comparison(time, accuracy,...)

In [ ]:
# Chossing dataset 
# Date
# Time [~m --> ~m]